# Data engenearing workbook

## Context:
My holdstaker needs to menage data from he's clients. 
I recived a pendrive with two main pathes, containg data from clients
Total archive first count:

In path 'NF' all files has the same prefix 'NF'
In path 'PlanilhaCorretores' file prefix are ['Prestação de Contas', 'Planilha Comissão']

## Taks:
### - Merge 'pdf' files from same client
      - Identify and Extract client name in all prefix
      - Join match files as Client Name.pdf

### - Create a table containing information about clients: name, processNumber, situation, responsableOne, nf(bool)
      - match = extract and compare name from prefix
      - name = originalName, processNumber = (get originalName and processNumber from table in pdf file)
      - situation,responsableOne = 'Planilha Comissão 'Name' (situation, responsableOne)
      - nf (for client in clients, if nf 'name' in nf[], return true, else, false)


### - Create a table containing information about cliente in NF: name, nf_number, date  

## 1 - Data Collect

In [ ]:
#1. Imports, Paths and Data Structures

import os
import re
import pandas as pd
import PyPDF2
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
import sys


PATH_NF = 'path_to_nf_files'
PATH_PLANILHAS_PRESTACOES = 'path_to_planilhas_prestacoes'

#data from NF
nf_table = {
    "origem": [],
    "nome_arquivo": [],
    "nome": [],
    "data": [],
    "corretor": [],
    "situação": []
}
#data from Prestação de Contas
pr_table = {
    "origem": [],
    "nome_arquivo": [],
    "nome": [],
    "nome_encontrado": [],
    "n_processo":[],
}
#data from Planilhas
pl_table = {
    "origem": [],
    "nome_arquivo": [],
    "nome": [],
    "corretor": [],
    "situação": []
}
#final data set - estrutura corrigida
final_table = {
    "nome": [],
    "origem_nf": [],      # Lista de caminhos de NF para este cliente
    "origem_pl": [],      # Lista de caminhos de Planilhas para este cliente
    "origem_pr": [],      # Lista de caminhos de Prestações para este cliente
    "data_nf": [],        # Data da NF
    "n_processo": [],     # Número do processo (da prestação)
    "situação": [],       # Situação (da planilha)
    "corretor": [],       # Corretor (da planilha)
    "nome_encontrado": [] # Nome encontrado no PDF da prestação
}


In [ ]:
#2. Populate nf_table, pr_table, pl_table with file_name and full paths

for fullPath in [PATH_PLANILHAS_PRESTACOES, PATH_NF]:
    for file in os.listdir(fullPath):
        if file.lower().startswith('nf '):
            nf_table["origem"].append(fullPath + '\\' + file)
            nf_table["nome_arquivo"].append(file)
        if file.lower().startswith('planilha'):
            pl_table["origem"].append(fullPath + '\\' + file)
            pl_table["nome_arquivo"].append(file)
        if file.lower().startswith('prestação'):
            pr_table["origem"].append(fullPath + '\\' + file)
            pr_table["nome_arquivo"].append(file)

print('Resumo da Análise de Arquivos PDF:')
print(f'Total de arquivos: {len(table["origem"])}')
print(f'Total de NF: {len(nf_table["origem"])}')
print(f'Total de Planilhas: {len(pl_table["origem"])}')
print(f'Total de Prestações: {len(pr_table["origem"])}')

In [ ]:
#3. Function to process names in all file_names as 'nome' , wich will be used as key for matching

def processar_nome(nome):
    nome = nome.title()

    # Remove prefix 
    for prefix in ['Planilha Comissão ', 'Prestação De Contas ', 'Nf ']:
        if nome.startswith(prefix.title()):
            nome = nome.replace(prefix.title(), '')

    # Remove sufix
    nome = nome.replace('.Pdf', '')
    nome = re.sub(r'\s*\(.*?\)', '', nome).strip()

    # Treating 'Espólio' cases
    if nome.lower().startswith(("espólio", "espolio")):
        nome = re.sub(r'^(Esp[óo]lio( De)?)\s*', '', nome, flags=re.IGNORECASE)
        nome = nome.strip() + " - Espólio"

    return nome

### 1.2 NF

#### Colect nf emission date in open file

In [ ]:
#1. Try colect data from NF files (FAILED ATTEMPT)

reader = PyPDF2.PdfReader(r"D:\Documentos\JOBS\DATA\RelatórioAndiappToledo\Toledo 2025_Edit\N Fiscais Toledo 2025\NF Abrahão Sadigursky.pdf")
texto = ""
for page in reader.pages:
    texto += page.extract_text()
if texto:
    print(texto)
else:
    print("No text found in the PDF.")

# No return in text extraction
# Suspecting that the PDF might be scanned images
# Need to use OCR to extract text from images in PDF

In [ ]:
#1.2 Try colect data from scanned PDF using OCR (SUCCESSFUL ATTEMPT)

# Tesseract executable path
TESSERACT_PATH = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = TESSERACT_PATH


def extrair_texto_pdf_ocr(file):
    texto_completo = ""
    try:
        # Abre o PDF
        doc = fitz.open(file)
        

        for i, pagina in enumerate(doc):
            # 1. Zoom para melhorar a qualidade da imagem para o OCR
            zoom = 2 
            matriz = fitz.Matrix(zoom, zoom)
            pix = pagina.get_pixmap(matrix=matriz)
            
            # 2. Converte para imagem PIL
            imagem = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            # 3. Tenta fazer o OCR
            try:
                # Tenta ler em PORTUGUÊS primeiro
                texto_pagina = pytesseract.image_to_string(imagem, lang='eng')
            except pytesseract.TesseractError as e:
                # Se der erro de "language not found", cai aqui
                print(f"\nAviso: {e}. Tentando OCR em INGLÊS.")

            texto_completo += texto_pagina
            

        doc.close()

    except Exception as e:
        print(f"\nERRO CRÍTICO: {e}")
        print("\nVerifique se o caminho na linha 9 está apontando para onde você instalou o Tesseract.")

    return texto_completo


def extrair_data_texto(texto):
    padrao_data = r"Data e Hora de Emiss[ãa]o.*?:?\s*\"?(\d{2}/\d{2}/\d{4})"
    match = re.search(padrao_data, texto, re.IGNORECASE | re.DOTALL)

    if match:
        return match.group(1)        
    else:
        print("N/A.")
        return None
        

    return data_encontrada

In [ ]:
#2. Populate nf_table with key names anda extracted dates

nf_table['nome'] = [processar_nome(n) for n in nf_table['nome_arquivo']]

for o in nf_table['origem']:
    texto = extrair_texto_pdf_ocr(o)
    data = extrair_data_texto(texto)    
    nf_table["data"].append(data)

In [ ]:
#3. Display nf_table as DataFrame
nf_df = pd.DataFrame(nf_table)
nf_df.head()

In [ ]:
#4. Display nf_table info
nf_df.info()

### 1.3 - Planilhas
#### Colect 'Corretor' and 'Situação' in file_name

In [ ]:
#1. Colect data from file_name as 'texto'

def extrair_situacao_corretor(texto):
    """
    Extrai situação e corretor do formato: (Situação Nome_Corretor)
    Exemplos: (VT João Silva), (V Total Maria Santos)
    """

    padrao = r'\(\s*(.+?)\s{2,}(.+?)\s*\)|' \
             r'\(\s*([A-Z]+)\s+([A-Z][^)]+)\)'
    
    match = re.search(padrao, texto)
    
    if match:
        # Tenta primeiro padrão (2+ espaços separam)
        if match.group(1) and match.group(2):
            return match.group(1).strip(), match.group(2).strip()
        # Se não, usa segundo padrão (situação em caps + nome)
        elif match.group(3) and match.group(4):
            return match.group(3).strip(), match.group(4).strip()
    
    print(f"Verificar formato em: {texto[:50]}...")
    return None, None

def extrair_situacao(texto):
    situacao, _ = extrair_situacao_corretor(texto)
    return situacao

def extrair_corretor(texto):
    _, corretor = extrair_situacao_corretor(texto)
    return corretor

In [ ]:
#2. Populate pl_table with key names, corretor and situação'
pl_table['nome'] = [processar_nome(n) for n in pl_table['nome_arquivo']]
pl_table['corretor'] = [extrair_corretor(t) for t in pl_table['origem']]
pl_table['situação'] = [extrair_situacao(t) for t in pl_table['origem']]

In [ ]:
#3. Display pl_table as DataFrame
pl_df = pd.DataFrame(pl_table)
pl_df.head()

In [ ]:
#4. Display pl_table info
pl_df.info()

### 1.4 - Prestação De Contas
#### Colect 'n_processo', 'nome_encontrado' in open file

In [ ]:
#1. Extract data from Prestação de Contas pdf files
from PyPDF2 import PdfReader

def extrair_dados_prestacao(caminho_pdf):
    """Extrai nome do cliente e número do processo de um arquivo PDF"""
    try:
        reader = PdfReader(caminho_pdf)
        texto = ''.join([page.extract_text() or '' for page in reader.pages])

        # Extrair nome do cliente
        nome_match = re.search(r'À\s+([^\n]+)', texto)
        nome_cliente = nome_match.group(1).strip() if nome_match else "Nome não encontrado"

        # Extrair número do processo (padrões variados)
        processo_match = re.search(r'(?:Processo|N° Processo)[:\s]*([\d\.\-/]+)', texto, re.IGNORECASE)
        numero_processo = processo_match.group(1).strip() if processo_match else "Número não encontrado"

        return nome_cliente, numero_processo

    except Exception as e:
        print(f"Erro ao processar {os.path.basename(caminho_pdf)}: {str(e)}")
        return "Erro no processamento", ""



pr_table['nome'] = [processar_nome(n) for n in pr_table['nome']]
pr_table['nome_encontrado'] = [extrair_dados_prestacao(o)[0] for o in pr_table['origem']]
pr_table['n_processo'] = [extrair_dados_prestacao(o)[1] for o in pr_table['origem']]

In [ ]:
#2. Populate pr_table with key names, nome_encontrado and n_processo
pr_table['nome'] = [processar_nome(n) for n in pr_table['nome']]
pr_table['nome_encontrado'] = [extrair_dados_prestacao(o)[0] for o in pr_table['origem']]
pr_table['n_processo'] = [extrair_dados_prestacao(o)[1] for o in pr_table['origem']]

In [ ]:
#3. Display pr_table as DataFrame
pr_df = pd.DataFrame(pr_table)
pr_df.head()

In [ ]:
#4. Display pr_table info
pr_df.info()

## Pipeline Final Table (Full Join)
Join data from 3 tables in final relatory

In [ ]:
# 1 Set names
todos_nomes = set()
todos_nomes.update(nf_table['nome'])
todos_nomes.update(pl_table['nome'])
todos_nomes.update(pr_table['nome'])

print(f"Total de clientes únicos encontrados: {len(todos_nomes)}")

In [ ]:
# 2. Group multiple origins by name
from collections import defaultdict

nf_por_nome = defaultdict(list)
pl_por_nome = defaultdict(list)
pr_por_nome = defaultdict(list)

# Agrupar NF por nome
for i, nome in enumerate(nf_table['nome']):
    nf_por_nome[nome].append({
        'origem': nf_table['origem'][i],
        'data': nf_table['data'][i]
    })

# Agrupar Planilhas por nome
for i, nome in enumerate(pl_table['nome']):
    pl_por_nome[nome].append({
        'origem': pl_table['origem'][i],
        'corretor': pl_table['corretor'][i],
        'situacao': pl_table['situação'][i]
    })

# Agrupar Prestações por nome
for i, nome in enumerate(pr_table['nome']):
    pr_por_nome[nome].append({
        'origem': pr_table['origem'][i],
        'nome_encontrado': pr_table['nome_encontrado'][i],
        'n_processo': pr_table['n_processo'][i]
    })

print("✓ Agrupamento por nome concluído")
print(f"  - Clientes com NF: {len(nf_por_nome)}")
print(f"  - Clientes com Planilha: {len(pl_por_nome)}")
print(f"  - Clientes com Prestação: {len(pr_por_nome)}")

In [ ]:
# 3 Construct final table with FULL JOIN

for nome in sorted(todos_nomes):
    # Adicionar nome
    final_table['nome'].append(nome)
    
    # ===== ORIGENS NF =====
    if nome in nf_por_nome:
        # Múltiplas NFs: adiciona lista de caminhos
        origens_nf = [nf['origem'] for nf in nf_por_nome[nome]]
        final_table['origem_nf'].append(origens_nf)
        
        # Data: pega da primeira NF (ou você pode concatenar todas)
        final_table['data_nf'].append(nf_por_nome[nome][0]['data'])
    else:
        final_table['origem_nf'].append(['Arquivo NF não encontrado'])
        final_table['data_nf'].append(None)
    
    # ===== ORIGENS PLANILHA =====
    if nome in pl_por_nome:
        # Múltiplas planilhas: adiciona lista de caminhos
        origens_pl = [pl['origem'] for pl in pl_por_nome[nome]]
        final_table['origem_pl'].append(origens_pl)
        
        # Situação e Corretor: pega da primeira (ou você pode verificar se são iguais)
        final_table['situação'].append(pl_por_nome[nome][0]['situacao'])
        final_table['corretor'].append(pl_por_nome[nome][0]['corretor'])
    else:
        final_table['origem_pl'].append(['Arquivo Planilha não encontrado'])
        final_table['situação'].append('Situação não encontrada')
        final_table['corretor'].append('Corretor não encontrado')
    
    # ===== ORIGENS PRESTAÇÃO =====
    if nome in pr_por_nome:
        # Múltiplas prestações: adiciona lista de caminhos
        origens_pr = [pr['origem'] for pr in pr_por_nome[nome]]
        final_table['origem_pr'].append(origens_pr)
        
        # Processo e nome encontrado: pega do primeiro
        final_table['n_processo'].append(pr_por_nome[nome][0]['n_processo'])
        final_table['nome_encontrado'].append(pr_por_nome[nome][0]['nome_encontrado'])
    else:
        final_table['origem_pr'].append(['Arquivo Prestação não encontrado'])
        final_table['n_processo'].append('Processo não encontrado')
        final_table['nome_encontrado'].append('Verificar prestação')

print(f"\n✓ Tabela final construída com {len(final_table['nome'])} clientes")

In [ ]:
# PASSO 4: Visualizar resumo e verificar inconsistências

# Converter para DataFrame para facilitar análise
df_final = pd.DataFrame(final_table)

print("="*60)
print("RESUMO DA TABELA FINAL")
print("="*60)
print(f"\nTotal de clientes: {len(df_final)}")
print(f"\nDimensões: {df_final.shape}")
print(f"\nColunas: {list(df_final.columns)}")

# Estatísticas de arquivos faltantes
print("\n" + "="*60)
print("ANÁLISE DE ARQUIVOS FALTANTES")
print("="*60)

nf_faltantes = df_final[df_final['origem_nf'].apply(lambda x: 'não encontrado' in str(x).lower())]
pl_faltantes = df_final[df_final['origem_pl'].apply(lambda x: 'não encontrado' in str(x).lower())]
pr_faltantes = df_final[df_final['origem_pr'].apply(lambda x: 'não encontrado' in str(x).lower())]

print(f"\n📄 Clientes SEM NF: {len(nf_faltantes)}")
print(f"📄 Clientes SEM Planilha: {len(pl_faltantes)}")
print(f"📄 Clientes SEM Prestação: {len(pr_faltantes)}")

# Clientes com múltiplos arquivos
multiplos_nf = df_final[df_final['origem_nf'].apply(lambda x: isinstance(x, list) and len(x) > 1 and 'não encontrado' not in str(x).lower())]
multiplos_pl = df_final[df_final['origem_pl'].apply(lambda x: isinstance(x, list) and len(x) > 1 and 'não encontrado' not in str(x).lower())]
multiplos_pr = df_final[df_final['origem_pr'].apply(lambda x: isinstance(x, list) and len(x) > 1 and 'não encontrado' not in str(x).lower())]

print("\n" + "="*60)
print("CLIENTES COM MÚLTIPLOS ARQUIVOS")
print("="*60)
print(f"\n📑 Clientes com >1 NF: {len(multiplos_nf)}")
if len(multiplos_nf) > 0:
    print("   Exemplos:")
    for idx in multiplos_nf.head(3).index:
        print(f"   - {df_final.loc[idx, 'nome']}: {len(df_final.loc[idx, 'origem_nf'])} arquivos")

print(f"\n📑 Clientes com >1 Planilha: {len(multiplos_pl)}")
if len(multiplos_pl) > 0:
    print("   Exemplos:")
    for idx in multiplos_pl.head(3).index:
        print(f"   - {df_final.loc[idx, 'nome']}: {len(df_final.loc[idx, 'origem_pl'])} arquivos")

print(f"\n📑 Clientes com >1 Prestação: {len(multiplos_pr)}")
if len(multiplos_pr) > 0:
    print("   Exemplos:")
    for idx in multiplos_pr.head(3).index:
        print(f"   - {df_final.loc[idx, 'nome']}: {len(df_final.loc[idx, 'origem_pr'])} arquivos")

# Mostrar primeiras linhas
print("\n" + "="*60)
print("PRIMEIRAS 10 LINHAS DA TABELA")
print("="*60)
print(df_final.head(10))

In [ ]:
# PASSO 5: Exportar tabela final para CSV/Excel

OUTPUT_FOLDER = r"D:\Documentos\JOBS\DATA\RelatórioAndiappToledo\Dados"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Exportar CSV
csv_path = os.path.join(OUTPUT_FOLDER, "tabela_final_clientes.csv")
df_final.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"✓ CSV exportado: {csv_path}")

# Exportar Excel (com formatação melhor para listas)
excel_path = os.path.join(OUTPUT_FOLDER, "tabela_final_clientes.xlsx")

# Converter listas para strings para visualização no Excel
df_export = df_final.copy()
for col in ['origem_nf', 'origem_pl', 'origem_pr']:
    df_export[col] = df_export[col].apply(lambda x: '\n'.join(x) if isinstance(x, list) else str(x))

df_export.to_excel(excel_path, index=False, engine='openpyxl')
print(f"✓ Excel exportado: {excel_path}")

# Gerar relatórios de clientes faltantes
relatorio_faltantes = {
    'nome': [],
    'falta_nf': [],
    'falta_pl': [],
    'falta_pr': [],
    'total_faltantes': []
}

for idx, row in df_final.iterrows():
    falta_nf = 'não encontrado' in str(row['origem_nf']).lower()
    falta_pl = 'não encontrado' in str(row['origem_pl']).lower()
    falta_pr = 'não encontrado' in str(row['origem_pr']).lower()
    
    total_faltantes = sum([falta_nf, falta_pl, falta_pr])
    
    if total_faltantes > 0:  # Só adiciona se faltar algo
        relatorio_faltantes['nome'].append(row['nome'])
        relatorio_faltantes['falta_nf'].append('SIM' if falta_nf else 'NÃO')
        relatorio_faltantes['falta_pl'].append('SIM' if falta_pl else 'NÃO')
        relatorio_faltantes['falta_pr'].append('SIM' if falta_pr else 'NÃO')
        relatorio_faltantes['total_faltantes'].append(total_faltantes)

df_faltantes = pd.DataFrame(relatorio_faltantes)
faltantes_path = os.path.join(OUTPUT_FOLDER, "clientes_arquivos_faltantes.xlsx")
df_faltantes.to_excel(faltantes_path, index=False, engine='openpyxl')
print(f"✓ Relatório de faltantes exportado: {faltantes_path}")

print(f"\n{'='*60}")
print(f"EXPORTAÇÃO CONCLUÍDA!")
print(f"{'='*60}")

In [ ]:
# NF Relatory - Match corretor and situação from pl_table to nf_table else return none

for nf in nf_table['nome']:
    encontrado = False
    for i, pl in enumerate(pl_table['nome']):
        if nf == pl:
            nf_table['corretor'].append(pl_table['corretor'][i])
            nf_table['situação'].append(pl_table['situação'][i])
            encontrado = True            
            break
    if not encontrado:
        nf_table['corretor'].append(None)

for k, v in nf_table.items():
    print(f"{k}: {v}")

# PDF Merge

In [ ]:
# 1 Check empty pages in each PDF in a folder and offer to delete them

def analisar_e_limpar_pdfs(caminho_da_pasta):
    """
    Analisa todos os arquivos PDF em uma pasta, relata suas propriedades
    e oferece a opção de remover páginas em branco.
    Filtra o relatório para exibir apenas arquivos com mais de uma página.

    Args:
        caminho_da_pasta (str): O caminho para a pasta contendo os arquivos PDF.
    """
    if not os.path.isdir(caminho_da_pasta):
        print(f"Erro: O caminho '{caminho_da_pasta}' não é uma pasta válida.")
        return

    # --- ESTRUTURAS DE DADOS PARA ARMAZENAR OS RESULTADOS ---
    relatorio_paginas = {}
    total_paginas_por_arquivo = {} # Dicionário auxiliar para guardar o total de páginas
    arquivos_com_mais_de_uma_pagina = 0
    arquivos_com_paginas_em_branco = {}

    print(f"--- Iniciando Análise da Pasta: {caminho_da_pasta} ---\n")

    lista_de_arquivos = [f for f in os.listdir(caminho_da_pasta) if f.lower().endswith('.pdf')]

    if not lista_de_arquivos:
        print("Nenhum arquivo PDF encontrado na pasta.")
        return

    # --- FASE 1: ANÁLISE DOS ARQUIVOS ---
    for nome_arquivo in lista_de_arquivos:
        caminho_completo = os.path.join(caminho_da_pasta, nome_arquivo)
        
        try:
            doc = fitz.open(caminho_completo)
            total_paginas = doc.page_count
            total_paginas_por_arquivo[nome_arquivo] = total_paginas # Armazena o total
            paginas_em_branco_indices = []
            
            # Verifica se o arquivo tem mais de uma página
            if total_paginas > 1:
                arquivos_com_mais_de_uma_pagina += 1

            # Itera por cada página para análise
            for i in range(total_paginas):
                page = doc.load_page(i)
                # Critério de página em branco: não tem texto e não tem imagens.
                if not page.get_text() and not page.get_images():
                    paginas_em_branco_indices.append(i)

            # Armazena a contagem de páginas que atendem ao critério (não estão em branco)
            paginas_validas = total_paginas - len(paginas_em_branco_indices)
            relatorio_paginas[nome_arquivo] = paginas_validas
            
            # Se encontrou páginas em branco, armazena quais são
            if paginas_em_branco_indices:
                arquivos_com_paginas_em_branco[nome_arquivo] = [p + 1 for p in paginas_em_branco_indices]

            doc.close()

        except Exception as e:
            print(f"Não foi possível processar o arquivo '{nome_arquivo}'. Erro: {e}")

    # --- FASE 2: APRESENTAÇÃO DOS RESULTADOS (COM FILTRO) ---
    print("--- Relatório da Análise Concluído ---\n")
    
    # MODIFICAÇÃO AQUI: Adicionado filtro para mostrar apenas arquivos com mais de 1 página
    print("1. Quantidade de páginas válidas (não-brancas) por arquivo (somente arquivos com >1 página):")
    if relatorio_paginas:
        arquivos_filtrados_exibidos = False
        for nome, qtd in relatorio_paginas.items():
            # A condição do filtro é aplicada aqui
            if total_paginas_por_arquivo.get(nome, 0) > 1:
                print(f"  - {nome}: {qtd} página(s)")
                arquivos_filtrados_exibidos = True
        if not arquivos_filtrados_exibidos:
             print("  Nenhum arquivo com mais de uma página foi encontrado para listar.")
    else:
        print("  Nenhum arquivo processado.")
    print("-" * 30)

    print(f"2. Total de arquivos com mais de uma página: {arquivos_com_mais_de_uma_pagina}")
    print("-" * 30)
    
    print("3. Arquivos que contêm páginas em branco:")
    if arquivos_com_paginas_em_branco:
        for nome, paginas in arquivos_com_paginas_em_branco.items():
            print(f"  - {nome} (Páginas em branco: {paginas})")
    else:
        print("  Nenhum arquivo com páginas em branco foi encontrado.")
    print("-" * 30)

    print("\n--- Verificação Manual Completa ---")


    print("\n-- ANÁLISE ---\n")
    print("Os arquivos com mais de uma página listados acima contêm informações válidas.")
    print("Os demais arquivos foram limpos de páginas em branco, se houveram.")
    
    # --- FASE 3: AÇÃO DE APAGAR PÁGINAS (COM CONFIRMAÇÃO) ---
    if not arquivos_com_paginas_em_branco:
        print("\nProcesso finalizado. Nenhuma ação de modificação necessária.")
        return

    prosseguir = input("\nDeseja apagar as páginas em branco listadas acima? (s/n): ").lower()

    if prosseguir == 's':
        print("\n--- Iniciando a remoção de páginas em branco ---")
        
        pasta_saida = os.path.join(caminho_da_pasta, "arquivos_limpos")
        os.makedirs(pasta_saida, exist_ok=True)
        print(f"Os arquivos modificados serão salvos em: '{pasta_saida}'")

        for nome_arquivo, paginas in arquivos_com_paginas_em_branco.items():
            try:
                caminho_original = os.path.join(caminho_da_pasta, nome_arquivo)
                doc = fitz.open(caminho_original)
                
                indices_para_remover = sorted([p - 1 for p in paginas], reverse=True)
                
                for indice in indices_para_remover:
                    doc.delete_page(indice)
                
                caminho_novo_arquivo = os.path.join(pasta_saida, nome_arquivo)
                doc.save(caminho_novo_arquivo, garbage=4, deflate=True)
                doc.close()
                print(f"  - Páginas removidas de '{nome_arquivo}'. Nova versão salva.")

            except Exception as e:
                print(f"  - Erro ao modificar o arquivo '{nome_arquivo}': {e}")
        
        print("\n--- Processo de limpeza concluído! ---")
    else:
        print("\nNenhuma modificação foi realizada. Processo encerrado.")


# --- COMO USAR O SCRIPT ---
if __name__ == "__main__":
    # IMPORTANTE: Substitua o caminho abaixo pela pasta onde estão seus PDFs.
    pasta_de_pdfs = "D:\Documentos\PyCharmD\.venv\Projetos 2025\ServiçoAnddiapToledo\Toledo 2025_Edit\Planilha Corretores 2025"
    
    analisar_e_limpar_pdfs(pasta_de_pdfs)



In [ ]:
# 2 Delete empty pages in each individual file in each directory

def apagar_ultima_pagina(diretorio, nome_arquivo):
    """
    Remove a última página de um arquivo PDF específico.
    Este método usa um arquivo temporário para garantir a compatibilidade e segurança,
    evitando o erro de "salvamento incremental".
    """
    caminho_completo = os.path.join(diretorio, nome_arquivo)
    
    # Define um nome para o arquivo temporário
    nome_temp = os.path.join(diretorio, "temp_" + nome_arquivo)

    # --- Verificações de Segurança ---
    if not os.path.isfile(caminho_completo):
        print(f"\nERRO: O arquivo '{nome_arquivo}' não foi encontrado no diretório especificado.")
        return

    try:
        # Abre o documento PDF original
        doc = fitz.open(caminho_completo)

        total_paginas = doc.page_count
        
        if total_paginas <= 1:
            print(f"\nAVISO: O arquivo '{nome_arquivo}' possui apenas uma página. Nenhuma ação foi tomada.")
            doc.close()
            return

        print(f"\nProcessando '{nome_arquivo}'...")
        print(f"O arquivo possui {total_paginas} páginas. A última página (nº {total_paginas}) será removida.")

        # Remove a última página
        doc.delete_page(total_paginas - 1)

        # --- NOVA LÓGICA DE SALVAMENTO ---
        # 1. Salva as alterações em um novo arquivo temporário
        doc.save(nome_temp, garbage=4, deflate=True)
        doc.close() # Fecha o documento original

        # 2. Remove o arquivo original antigo
        os.remove(caminho_completo)

        # 3. Renomeia o arquivo temporário para o nome do original
        os.rename(nome_temp, caminho_completo)

        print(f"\nSUCESSO! A última página do arquivo '{nome_arquivo}' foi removida.")

    except Exception as e:
        print(f"\nERRO: Ocorreu um problema ao processar o arquivo '{nome_arquivo}': {e}")
        # Se um erro ocorrer, verifica se um arquivo temporário ficou para trás e o remove
        if os.path.exists(nome_temp):
            os.remove(nome_temp)


# --- PONTO DE EXECUÇÃO PRINCIPAL ---
if __name__ == "__main__":
    # 1. CONFIGURE O DIRETÓRIO AQUI
    # IMPORTANTE: Substitua o caminho abaixo pelo caminho ABSOLUTO da pasta dos seus PDFs.
    # Exemplo Windows: dir = "C:\\Users\\SeuUsuario\\Desktop\\Meus Documentos"
    # Exemplo Linux/Mac: dir = "/home/seu_usuario/documentos/pdfs_para_limpar"
    dir = r"D:\Documentos\PyCharmD\.venv\Projetos 2025\ServiçoAnddiapToledo\Toledo 2025_Edit\Planilha Corretores 2025"
    
    # Verifica se o caminho foi alterado
    if "coloque/o/caminho" in dir:
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("!!! ATENÇÃO: Você precisa editar o script e configurar a     !!!")
        print("!!! variável 'dir' com o caminho para a sua pasta de PDFs.  !!!")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    else:
        # 2. PEÇA PARA O USUÁRIO INFORMAR O NOME DO ARQUIVO
        print("--- Ferramenta para Remover a Última Página de um PDF ---")
        print(f"Procurando arquivos no diretório: {dir}")
        print("AVISO: Esta ação irá modificar o arquivo original. Tenha um backup.\n")
        
        nome_do_arquivo = input("Informe o nome do arquivo PDF (ex: relatorio.pdf): ")

        # 3. CHAME A FUNÇÃO PARA FAZER A LIMPEZA
        if nome_do_arquivo:
            apagar_ultima_pagina(dir, nome_do_arquivo)
        else:
            print("Nenhum nome de arquivo foi inserido. Encerrando o programa.")

In [ ]:
# 3 Merge cleaned PDFs per client

# Definir pasta de destino
PASTA_DESTINO = r"OUTPUT"
os.makedirs(PASTA_DESTINO, exist_ok=True)

print("="*60)
print("INICIANDO MESCLAGEM DE PDFs POR CLIENTE")
print("="*60)

contador_sucesso = 0
contador_erro = 0
clientes_com_erro = []

for idx, row in df_final.iterrows():
    nome_cliente = row['nome']
    
    try:
        merger = PyPDF2.PdfMerger()
        arquivos_adicionados = 0
        
        # ORDEM: 1. Prestação, 2. Planilha, 3. NF
        ordem_merge = [
            ('Prestação', row['origem_pr']),
            ('Planilha', row['origem_pl']),
            ('NF', row['origem_nf'])
        ]
        
        for tipo, origens in ordem_merge:
            # Verificar se não é arquivo faltante
            if isinstance(origens, list) and 'não encontrado' not in str(origens).lower():
                # Adicionar todos os arquivos deste tipo (suporta múltiplos)
                for origem in origens:
                    if os.path.exists(origem):
                        merger.append(origem)
                        arquivos_adicionados += 1
                        print(f"  ✓ {tipo}: {os.path.basename(origem)}")
                    else:
                        print(f"  ⚠ {tipo}: arquivo não existe - {origem}")
        
        # Só salvar se adicionou pelo menos um arquivo
        if arquivos_adicionados > 0:
            caminho_saida = os.path.join(PASTA_DESTINO, f"{nome_cliente}.pdf")
            merger.write(caminho_saida)
            merger.close()
            print(f"✓ PDF mesclado criado: {nome_cliente}.pdf ({arquivos_adicionados} arquivos)\n")
            contador_sucesso += 1
        else:
            print(f"⚠ Nenhum arquivo encontrado para {nome_cliente} - PDF não criado\n")
            clientes_com_erro.append(nome_cliente)
            contador_erro += 1
            merger.close()
            
    except Exception as e:
        print(f"❌ ERRO ao processar {nome_cliente}: {e}\n")
        clientes_com_erro.append(nome_cliente)
        contador_erro += 1

# Final Relatory

In [ ]:
print("="*60)
print("RESUMO DA MESCLAGEM")
print("="*60)
print(f"✓ PDFs criados com sucesso: {contador_sucesso}")
print(f"❌ Clientes com erro/sem arquivos: {contador_erro}")
print(f"\nPasta de destino: {PASTA_DESTINO}")

if clientes_com_erro:
    print(f"\n⚠ Clientes que não geraram PDF:")
    for cliente in clientes_com_erro[:10]:  # Mostrar primeiros 10
        print(f"  - {cliente}")
    if len(clientes_com_erro) > 10:
        print(f"  ... e mais {len(clientes_com_erro) - 10} clientes")

print("="*60)